In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Importing libraries 
train_data = pd.read_csv("../input/analyticsclubproject/train.csv")
test_data = pd.read_csv("../input/analyticsclubproject/test.csv")
train_data.info()

In [ ]:
train_data.head()

In [ ]:
train_data['Debt'] = train_data['Debt_To_Income']*train_data['Annual_Income']
corr_mat = train_data.corr()
corr_mat['Interest_Rate'].sort_values(ascending = False)

In [ ]:
from pandas.plotting import scatter_matrix
attributes = ['Inquiries_Last_6Mo','Debt_To_Income','Loan_Amount_Requested','Length_Employed','Number_Open_Accounts','Total_Accounts','Annual_Income','Debt']
scatter_matrix(train_data[attributes], figsize = (20,20))

In [ ]:
X = train_data.copy()
y = train_data['Interest_Rate'].copy()

In [ ]:
from sklearn.preprocessing import StandardScaler
to_drop = ['Loan_ID','Interest_Rate']
X['Annual_Income'] = X['Annual_Income'].replace(-1,X['Annual_Income'].mean())
X.drop(to_drop, axis = 'columns', inplace = True)
ss = StandardScaler()
ss.fit_transform(X)

In [ ]:
from sklearn.model_selection import cross_val_score
import lightgbm as lgb
clf = lgb.LGBMClassifier(learning_rate=0.125, max_bin=255, num_iterations=70,n_estimators = 600)
# gbc = XGBClassifier(n_estimators = 500, learning_rate = 0.1)
scores = cross_val_score(clf,X,y, scoring='accuracy',cv = 5)
scores

In [ ]:
print('mean_score:', scores.mean())
print('std: ',scores.std())

In [ ]:
clf.fit(X,y)
feature_importance = clf.feature_importances_ 
sorted(zip(feature_importance,X.columns))

In [ ]:
# # GridSearch 
# from sklearn.model_selection import GridSearchCV
# param_grid = [{'learning_rate':[0.4,0.45,0.5,0.55,0.6],'max_bin':[375,350,325,300,275,255,225],'num_iterations':[70,80,90,95,100,105,110]}]
# clf_2 = lgb.LGBMClassifier()
# grid_search = GridSearchCV(clf_2, param_grid, cv = 5, scoring = 'accuracy', return_train_score= True)
# grid_search.fit(X,y)

In [ ]:
# saving models:
# import joblib
# joblib.dump(gbc,"model_gbc.pkl")
# for loading:
# gbc = joblib.load("model_gbc.pkl")

In [ ]:
# grid_search.best_params_

In [ ]:
# grid_search.best_estimator_

GridSearchCV(cv=5, estimator=LGBMClassifier(),
             param_grid=[{'learning_rate': [0.1,0.4, 0.45, 0.5, 0.55, 0.6],
                          'max_bin': [375, 350, 325, 300, 275, 255, 225],
                          'num_iterations': [70, 80, 90, 95, 100, 105, 110]}],
             return_train_score=True, scoring='accuracy')


grid_search.best_params_
{'learning_rate': 0.1, 'max_bin': 255, 'num_iterations': 70}


grid_search.best_estimator_
LGBMClassifier(learning_rate=0.4, max_bin=255, num_iterations=70)



In [ ]:
test_data['Debt'] = test_data['Debt_To_Income']*test_data['Annual_Income']

In [ ]:
final_pred = clf.predict(test_data.drop(['Loan_ID'], axis = 'columns'))
submission = pd.DataFrame({'Loan_ID': test_data['Loan_ID'], 'Interest_Rate': final_pred}).set_index('Loan_ID')
print(submission)
submission.to_csv('submission.csv')